In [39]:
import pandas as pd
import numpy as np
import datetime

main_path = '/Users/PC/Desktop/Insight/MTASafety'

In [41]:
ModelData=pd.read_csv(main_path + '/MergedDataUse.csv', low_memory=False)

In [42]:
ModelData.shape

(2310745, 95)

In [43]:
ModelData['datetime_clean_x'] = pd.to_datetime(ModelData['datetime_clean_x'])

In [44]:
#Sorting by Date
ModelData=ModelData.sort_values(by=['datetime_clean_x'])

In [45]:
#Creating the Day/Time Identity Variable that I'll Group By
ModelData['Ident2']= ModelData['Station2'].astype(str) + ModelData['wkdaynbr_x'].astype(str) + ModelData['timegroup_x'].astype(str)

In [46]:
#Making strings out of the offenses
ModelData['OFNS_DESC_x']=ModelData['OFNS_DESC_x'].astype(str)
ModelData['LAW_CAT_CD_x']=ModelData['LAW_CAT_CD_x'].astype(str)

In [47]:
#Creating Training Data (2017-2018)
Training = ModelData[ModelData['datetime_clean_x'] < '2019-01-01']

In [48]:
#Merging Down Crimes Entries and Exits
ident_sumsTrain=Training.groupby('Ident2')['Complaint_x','entries_diff_x','exit_diff_x'].sum().reset_index()
everything_elseTrain = Training.groupby('Ident2').first()

In [49]:
#Adding the Crimes Description and Offense Type to Training Data for Additional Use
traincrimes = Training.groupby(['Ident2'])['OFNS_DESC_x'].apply(', '.join).reset_index()
trainofftype = Training.groupby(['Ident2'])['LAW_CAT_CD_x'].apply(', '.join).reset_index()
TrainingMerge = pd.merge(ident_sumsTrain, traincrimes, on=['Ident2'], how='left')
TrainingMerge2 = pd.merge(TrainingMerge, trainofftype, on=['Ident2'], how='left')
Training2 = pd.merge(TrainingMerge2, everything_elseTrain, on=['Ident2'], how='left')

In [50]:
#Combining Entries and Exits
Training2["CombinedEntriesExits"] = (Training2["entries_diff_x_x"] + Training2["exit_diff_x_x"])

In [51]:
#Creating Rate of Crime
Training2["RateofCrime"] = Training2["Complaint_x_x"]/Training2["CombinedEntriesExits"]

In [52]:
#Setting all Infinites to 0
Training2.loc[~np.isfinite(Training2['RateofCrime']), 'RateofCrime'] = 0

In [53]:
#Creating a Metric for Just Entries As Well
Training2["RateofCrimeJustEntry"] = Training2["Complaint_x_x"]/Training2["entries_diff_x_x"]
Training2.loc[~np.isfinite(Training2['RateofCrimeJustEntry']), 'RateofCrimeJustEntry'] = 0

In [54]:
#Creating Test Data(2019, First 3 Months)
Testing = ModelData[ModelData['datetime_clean_x'] > '2018-12-31']

In [55]:
#Creating Smaller Test Set Grouped by Day, Hour
ident_sumsTesting=Testing.groupby('Ident2')['Complaint_x','entries_diff_x','exit_diff_x'].sum().reset_index()

In [56]:
#Adding the Crime Descriptions and Types to the Testing Set
testcrimes = Testing.groupby(['Ident2'])['OFNS_DESC_x'].apply(', '.join).reset_index()
testofftype = Testing.groupby(['Ident2'])['LAW_CAT_CD_x'].apply(', '.join).reset_index()
everything_elseTesting = Testing.groupby('Ident2').first()
TestingMerge = pd.merge(ident_sumsTesting, testcrimes, on=['Ident2'], how='left')
TestingMerge2 = pd.merge(TestingMerge, testofftype, on=['Ident2'], how='left')
Testing2 = pd.merge(TestingMerge2, everything_elseTesting, on=['Ident2'], how='left')
Testing2["CombinedEntriesExits"] = (Testing2["entries_diff_x_x"] + Testing2["exit_diff_x_x"])
Testing2["RateofCrime"] = Testing2["Complaint_x_x"]/Testing2["CombinedEntriesExits"]
Testing2.loc[~np.isfinite(Testing2['RateofCrime']), 'RateofCrime'] = 0

In [57]:
#Creating a Separate Just Entry Rate of Crime for Testing Set
Testing2["RateofCrimeJustEntry"] = Testing2["Complaint_x_x"]/Testing2["entries_diff_x_x"]
Testing2.loc[~np.isfinite(Testing2['RateofCrimeJustEntry']), 'RateofCrimeJustEntry'] = 0

In [58]:
Training2.shape

(20118, 104)

In [59]:
Testing2.shape

(20118, 104)

In [ ]:
#Saving the Datasets
Training2.to_csv(main_path + '/Training6_20.csv')
Testing2.to_csv(main_path + '/Testing6_20.csv')